In [1]:
#设置Tensorflow
import os
os.environ["OMP_NUM_THREADS"]="7"
import pandas as pd
import tensorflow as tf

In [11]:
#加载数据集
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train,x_test=x_train/255.0,x_test/255.0 #将样本数据从整数转换为浮点数

In [3]:
#通过堆叠层来构建机器学习模型
model=tf.keras.models.Sequential([
tf.keras.layers.Flatten(input_shape=(28,28)),#该层次将数据从二维数组展平成一维数组
tf.keras.layers.Dense(128,activation="relu"),#这是一个全连接层，具有128个神经元和ReLU激活函数
tf.keras.layers.Dropout(0.2),#这个层在训练过程中随机丢弃一部分神经元的激活值，丢弃比例为20%(0.2),以减少过拟合
tf.keras.layers.Dense(10)#模型的输出层，有10个神经元，这通常对应于分类任务中的类别数量
])#线性堆叠的层次模型，用于快速构建一个前馈神经网络

c:\Users\youyiting\anaconda3\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.13478138, -0.03413622, -0.5517986 , -0.35491657, -0.24875826,
        -1.0369924 , -0.5020624 ,  0.4550425 , -0.2794225 ,  0.69409925]],
      dtype=float32)

In [6]:
#将logits转换为每个类的概率
tf.nn.softmax(predictions).numpy()

array([[0.12093743, 0.10214116, 0.06086714, 0.07411186, 0.08241223,
        0.03746846, 0.06397099, 0.16658981, 0.07992348, 0.21157749]],
      dtype=float32)

使用 losses.SparseCategoricalCrossentropy 为训练定义损失函数，它会接受 

logits 向量和 True 索引，并为每个样本返回一个标量损失。

In [7]:

loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [9]:
loss_fn(y_train[:1], predictions).numpy()

3.284256

In [10]:
#配置和编译模型
model.compile(optimizer="adam",
              loss=loss_fn,
              metrics=["accuracy"])

In [14]:
#训练和评估模型
model.fit(x_train,y_train,epochs=5)
model.evaluate(x_test,y_test,verbose=2)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 946us/step - accuracy: 0.9784 - loss: 0.0686
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 933us/step - accuracy: 0.9819 - loss: 0.0568
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 937us/step - accuracy: 0.9836 - loss: 0.0493
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 987us/step - accuracy: 0.9843 - loss: 0.0476
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 966us/step - accuracy: 0.9854 - loss: 0.0440
313/313 - 0s - 624us/step - accuracy: 0.9794 - loss: 0.0715


[0.07149577885866165, 0.9793999791145325]

In [15]:
#如果想让模型返回概率值,可以封装经过训练的模型，并将softmax附加到该模型
probability_model=tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.84347682e-09, 5.45483880e-10, 2.25658880e-07, 2.84735252e-05,
        8.91369281e-13, 3.54000695e-09, 1.48560960e-17, 9.99971151e-01,
        6.95553448e-10, 1.36723159e-07],
       [5.94566443e-13, 1.49390260e-07, 9.99999642e-01, 2.42805527e-07,
        7.86597306e-21, 1.41632206e-09, 4.55461580e-12, 1.55312815e-19,
        1.30194300e-09, 5.26332388e-22],
       [2.44133402e-09, 9.99902248e-01, 5.66152630e-06, 4.86098656e-07,
        1.54220771e-07, 2.21087447e-07, 3.32548507e-06, 6.29887509e-05,
        2.47845528e-05, 4.50443594e-09],
       [9.99272764e-01, 1.58576541e-10, 6.68505572e-06, 9.48077616e-09,
        2.37134699e-07, 3.58454599e-07, 7.14181457e-04, 3.64956527e-08,
        6.30592778e-09, 5.81887298e-06],
       [1.19908954e-08, 7.76896741e-11, 6.28069197e-09, 1.08912435e-10,
        9.98656631e-01, 6.40681552e-09, 2.00828197e-07, 1.55512625e-05,
        1.14828955e-08, 1.32760825e-03]], dtype=float32)>